In [ ]:
import os
import time
import logging
from dataclasses import dataclass
from typing import Optional

from openai import OpenAI

# ----------------------------
# Logging (production-friendly)
# ----------------------------
logging.basicConfig(
    level=os.getenv("LOG_LEVEL", "INFO"),
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
)
log = logging.getLogger("gemini-client")


# ----------------------------
# Config
# ----------------------------
@dataclass(frozen=True)
class Settings:
    # Gemini OpenAI-compatible endpoint (AI Studio / Generative Language API)
    base_url: str = "https://generativelanguage.googleapis.com/v1beta/openai/"
    api_key: str = os.getenv("GEMINI_API_KEY", "")
    model: str = os.getenv("MODEL", "gemini-2.5-flash")

    # Reliability controls
    timeout_seconds: int = int(os.getenv("TIMEOUT_SECONDS", "30"))
    max_retries: int = int(os.getenv("MAX_RETRIES", "3"))
    retry_backoff_seconds: float = float(os.getenv("RETRY_BACKOFF_SECONDS", "1.5"))


def build_client(settings: Settings) -> OpenAI:
    if not settings.api_key:
        raise ValueError(
            "Missing GEMINI_API_KEY. Set it as an environment variable."
        )

    # OpenAI SDK client pointed to Gemini
    return OpenAI(
        api_key=settings.api_key,
        base_url=settings.base_url,
        timeout=settings.timeout_seconds,
    )


# ----------------------------
# Core wrapper (single place to call the model)
# ----------------------------
class LLMService:
    def __init__(self, settings: Settings):
        self.settings = settings
        self.client = build_client(settings)

    def chat(self, user_text: str, system_text: str = "You are a helpful assistant.") -> str:
        last_error: Optional[Exception] = None

        for attempt in range(1, self.settings.max_retries + 1):
            try:
                resp = self.client.chat.completions.create(
                    model=self.settings.model,
                    messages=[
                        {"role": "system", "content": system_text},
                        {"role": "user", "content": user_text},
                    ],
                )
                return resp.choices[0].message.content or ""

            except Exception as e:
                last_error = e
                log.warning("LLM call failed (attempt %s/%s): %s",
                            attempt, self.settings.max_retries, repr(e))
                if attempt < self.settings.max_retries:
                    sleep_for = self.settings.retry_backoff_seconds * attempt
                    time.sleep(sleep_for)

        # If all retries fail, raise a clean error
        raise RuntimeError(f"LLM call failed after retries: {last_error!r}")


# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    settings = Settings()
    llm = LLMService(settings)

    prompt = "Explain to me how AI works in simple terms."
    answer = llm.chat(prompt)

    print("\n--- MODEL ANSWER ---\n")
    print(answer)
